# Slack

This notebook shows how to use the Slack chat loader. This class helps map exported slack conversations to LangChain chat messages.

The process has three steps:
1. Export the desired conversation thread by following the [instructions here](https://slack.com/help/articles/1500001548241-Request-to-export-all-conversations).
2. Create the `SlackChatLoader` with the file path pointed to the json file or directory of JSON files
3. Call `loader.load()` (or `loader.lazy_load()`) to perform the conversion.

## 1. Creat message dump

Currently (2023/08/23) this loader best supports a zip directory of files in the format generated by exporting your a direct message converstion from Slack. Follow up-to-date instructions from slack on how to do so.

We have an example in the LangChain repo.

In [16]:
import requests

permalink = "https://raw.githubusercontent.com/langchain-ai/langchain/342087bdfa3ac31d622385d0f2d09cf5e06c8db3/libs/langchain/tests/integration_tests/examples/slack_export.zip"
response = requests.get(permalink)
with open("slack_dump.zip", "wb") as f:
    f.write(response.content)

## 2. Create the Chat Loader

Provide the loader with the file path to the zip directory. You can optionally specify the user id that maps to an ai message as well an configure whether to merge message runs.

In [17]:
from langchain.chat_loaders.slack import SlackChatLoader

In [18]:
loader = SlackChatLoader(
    zip_path="slack_dump.zip",
    user_id="U0500003428",
)

## 3. Load messages

The `load()` (or `lazy_load`) methods return a list of "ChatSessions" that currently just contain a list of messages per loaded conversation.

In [21]:
from typing import List
from langchain.chat_loaders.base import ChatSession

messages: List[ChatSession] = loader.load()
messages

[{'messages': []},
 {'messages': [AIMessage(content='Example message', additional_kwargs={'sender': 'U0500003428', 'events': [{'message_time': '1681413829.513479'}]}, example=False)]},
 {'messages': [AIMessage(content='Hello, world!\n\nExample in general', additional_kwargs={'sender': 'U0500003428', 'events': [{'message_time': '1681413317.397479'}, {'message_time': '1681413834.502859'}]}, example=False)]},
 {'messages': []},
 {'messages': [AIMessage(content='Example random message', additional_kwargs={'sender': 'U0500003428', 'events': [{'message_time': '1681413852.964959'}]}, example=False)]},
 {'messages': []}]

### Next Steps

You can then use these messages how you see fit, such as finetuning a model, few-shot example selection, or directly make predictions for the next message  

In [23]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[1]['messages']):
    print(chunk.content, end="", flush=True)

Hi [Friend's Name],

I hope you're doing well! It's been a while since we last caught up, and I wanted to see how you've been doing. How are things going with work/school? Have you been up to anything exciting lately?

I recently took a trip to [destination] and had an amazing time exploring the new city. The architecture was stunning and the food was delicious. I also started a new hobby of painting, and it's been a great way to relax and express my creativity.

I've been thinking about planning a get-together with our group of friends. It would be so nice to catch up and spend some quality time together. Let me know if you're interested and we can start coordinating a date and location.

Anyway, I hope to hear from you soon! Take care and let's catch up soon.

Best,
[Your Name]